<a href="https://colab.research.google.com/github/KatrinaH92/git_test/blob/main/Decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using decision tree

In [69]:
import numpy as np
import tensorflow as tf
import json
import scipy

import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, metrics, Sequential, utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

from tensorflow.keras.layers import Dense, Embedding, LSTM
import pandas as pd
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [47]:
BATCH_SIZE = 512

In [48]:
METRICS = [
      lambda : tf.keras.metrics.TruePositives(name='tp'),
      lambda : tf.keras.metrics.FalsePositives(name='fp'),
      lambda : tf.keras.metrics.TrueNegatives(name='tn'),
      lambda : tf.keras.metrics.FalseNegatives(name='fn'),

      lambda : tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      lambda : tf.keras.metrics.Precision(name='precision'),
      lambda : tf.keras.metrics.Recall(name='recall'),
]

def fresh_metrics():
    return [metric() for metric in METRICS]

In [49]:
data = pd.read_csv(r'Libs cons.csv')

In [54]:
# Load and preprocess data
features = data[['Title', 'Text']].copy()
features.fillna("", inplace=True)
features['combined_text'] = features['Title'] + " " + features['Text']

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(data['Political Lean'])

# 🚨 Fit TfidfVectorizer on the ENTIRE dataset first
vectorizer = TfidfVectorizer(max_features=100)  # Limit features to 100
X = vectorizer.fit_transform(features['combined_text'])  # Fit on full dataset

# ✅ Now split AFTER vectorization to keep feature consistency
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

In [51]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, one_hot_labels, test_size=0.2, random_state=42
)


In [52]:
X_train = pd.DataFrame(X_train)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2207,7266,1377,1014
1,0,0,0,0,0,0,0,0,0,0,...,39,3055,8,46,82,2364,22,13,8293,118
2,0,0,0,0,0,0,0,0,0,0,...,5,4081,4,6689,2735,40,79,1230,6,52
3,0,0,0,0,0,0,0,0,0,0,...,1294,1,1461,3,267,1352,76,1124,12,13
4,0,0,0,0,0,0,0,0,0,0,...,101,1070,661,242,5,3267,429,26,503,3273


In [74]:
# Train RandomForest Model
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Train the model on the resampled data
model = XGBClassifier(n_estimators=800, max_depth=15, random_state=42)
model.fit(X_train_res, y_train_res)
model.score(X_test, y_test)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Training Accuracy: 0.89
Test Accuracy: 0.68

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.25      0.36       913
           1       0.69      0.91      0.79      1658

    accuracy                           0.68      2571
   macro avg       0.65      0.58      0.57      2571
weighted avg       0.66      0.68      0.63      2571

